In [2]:
# === IMPORTS ===
import pandas as pd
import time
import numpy as np

# === CONSTANTS ===
GT = 14052  # Gross tonnage of the ship
FUEL_DENSITY = 0.991  # kg/L
CO2_FACTOR = 3.114  # g CO2 per gram of HFO

# === FEATURES USED FOR CALCULATION ===
features = [
    'FO_ME_Cons', 'FO_GE_Cons', 'Ship_Speed', 'CppPitch', 'Wind_Speed',
    'HEEL', 'Fore_Draft', 'Aft_Draft'
]

# === INPUT CSV FILE ===
csv_path = input("Enter path to minute-wise CSV file: ").strip()
data = pd.read_csv(csv_path)

# === CLEAN AND PREPROCESS FUNCTION ===
def preprocess(row, prev_row):
    try:
        # Convert fuel readings to numeric and compute fuel diff
        fuel_me = max(float(row['FO_ME_Cons']) - float(prev_row['FO_ME_Cons']), 0)
        fuel_ge = max(float(row['FO_GE_Cons']) - float(prev_row['FO_GE_Cons']), 0)
        fuel_liters = fuel_me + fuel_ge

        # Trim and average draft
        trim = float(row['Aft_Draft']) - float(row['Fore_Draft'])
        avg_draft = (float(row['Aft_Draft']) + float(row['Fore_Draft'])) / 2

        # CO2 and Distance
        ship_speed = float(row['Ship_Speed'])
        distance_nm = ship_speed / 60  # per minute
        fuel_kg = fuel_liters * FUEL_DENSITY
        co2_emission_g = fuel_kg * 1000 * CO2_FACTOR

        # Instantaneous CII Calculation
        if GT * distance_nm == 0:
            cii = np.nan
        else:
            cii = co2_emission_g / (GT * distance_nm)

        return {
            'CO2_Emission_g': co2_emission_g,
            'Distance_NM': distance_nm,
            'CII': cii,
            'Avg_Speed': ship_speed,
            'Avg_Trim': trim,
            'Avg_Heel': float(row['HEEL']),
            'Avg_Wind_Speed': float(row['Wind_Speed']),
            'Avg_CppPitch': float(row['CppPitch']),
            'Avg_Draft': avg_draft
        }
    except Exception as e:
        print(f"Error processing row: {e}")
        return {
            'CO2_Emission_g': np.nan,
            'Distance_NM': np.nan,
            'CII': np.nan,
            'Avg_Speed': np.nan,
            'Avg_Trim': np.nan,
            'Avg_Heel': np.nan,
            'Avg_Wind_Speed': np.nan,
            'Avg_CppPitch': np.nan,
            'Avg_Draft': np.nan
        }

# === OPTIMIZATION SUGGESTION FUNCTION ===
def generate_suggestion(row):
    suggestions = []
    if row['Avg_Trim'] > 1.0:
        suggestions.append("Reduce trim to improve fuel efficiency")
    if row['Avg_Heel'] > 0.5:
        suggestions.append("Balance ballast to reduce heel")
    if row['Avg_Wind_Speed'] > 12:
        suggestions.append("Avoid sailing during high wind")
    if row['Avg_CppPitch'] < 15:
        suggestions.append("Increase CPP pitch for propulsion efficiency")
    if row['Avg_Speed'] < 17:
        suggestions.append("Maintain optimal cruising speed")
    return "; ".join(suggestions) if suggestions else "Performance is within expected range"

# === LIVE SIMULATION ===
print("\n--- LIVE CII CALCULATION STARTED ---")
for i in range(1, len(data)):
    current_row = data.iloc[i]
    previous_row = data.iloc[i - 1]

    result = preprocess(current_row, previous_row)

    cii = result.get('CII', np.nan)
    suggestions = generate_suggestion(result)

    if not np.isnan(cii):
        print(f"Minute {i}: Instant CII = {cii:.4f} | Suggestions: {suggestions}")
    else:
        print(f"Minute {i}: CII could not be calculated due to zero distance.")

    time.sleep(1)

print("\n--- LIVE CII SIMULATION COMPLETE ---")

C:\Users\siddh\AppData\Local\Temp\ipykernel_20932\560856955.py:19: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(csv_path)



--- LIVE CII CALCULATION STARTED ---
Minute 1: Instant CII = 19.1521 | Suggestions: Performance is within expected range
Minute 2: Instant CII = 18.3860 | Suggestions: Performance is within expected range
Minute 3: Instant CII = 17.6200 | Suggestions: Balance ballast to reduce heel
Minute 4: Instant CII = 17.6200 | Suggestions: Performance is within expected range
Minute 5: Instant CII = 16.0878 | Suggestions: Performance is within expected range
Minute 6: Instant CII = 17.6200 | Suggestions: Performance is within expected range
Minute 7: Instant CII = 17.6200 | Suggestions: Performance is within expected range
Minute 8: Instant CII = 17.7230 | Suggestions: Performance is within expected range
Minute 9: Instant CII = 18.3860 | Suggestions: Avoid sailing during high wind
Minute 10: Instant CII = 18.3860 | Suggestions: Performance is within expected range
Minute 11: Instant CII = 22.3464 | Suggestions: Performance is within expected range
Minute 12: Instant CII = 24.5147 | Suggestions: 

KeyboardInterrupt: 